In [1]:
import numpy as np
import pandas as pd
%matplotlib qt
import matplotlib.pyplot as plt

from modelbase.ode import Simulator


Could not find GLIMDA.


In [2]:
%%capture
cd ..

In [3]:
from models import get_model


m = get_model()


def get_new_y0(s):
    return dict(s.get_results_df().iloc[-1])

In [4]:
y0 = {
    "PQ": 11.027139850905353,
    "PC": 1.8895071932002812,
    "Fd": 3.8690237263896705,
    "ATP": 1.620195002854852,
    "NADPH": 0.4882103700673736,
    "H": 0.0022147075094596015,
    "LHC": 0.8023074419510501,
    "Psbs": 0.9607146039898598,
    "Vx": 0.950783616933656,
    "PGA": 0.9913970817549008,
    "BPGA": 0.0005355311557548053,
    "GAP": 0.0062630116252017295,
    "DHAP": 0.13778623933075737,
    "FBP": 0.006126990841013743,
    "F6P": 0.31166103888161867,
    "G6P": 0.7168203893211117,
    "G1P": 0.041575582577936025,
    "SBP": 0.01311315151803723,
    "S7P": 0.15782894767619207,
    "E4P": 0.00732079113061801,
    "X5P": 0.022396849486562384,
    "R5P": 0.03751472214765548,
    "RUBP": 0.13153657267999222,
    "RU5P": 0.015005888732707041,
    "MDA": 5.85270097771621e-06,
    "ASC": 9.999994138785292,
    "H2O2": 3.4273920330125316e-06,
    "DHA": 8.513863740903352e-09,
    "GSH": 9.999999991725186,
    "GSSG": 4.137406632226743e-09,
    "TR_ox": 0.9,
    "E_inactive": 4.7368421052631575,
}

y0_1000 = {
    "PQ": 1.507789886702025,
    "PC": 3.953187158958296,
    "Fd": 1.1279937081043432,
    "ATP": 2.400518163834891,
    "NADPH": 0.7833511016622213,
    "H": 11.834462552026038,
    "LHC": 0.3057293990674661,
    "Psbs": 0.5000000000384304,
    "Vx": 0.09090909091242916,
    "PGA": 0.04883787233687633,
    "BPGA": 0.0002431280086079331,
    "GAP": 0.019630698252342628,
    "DHAP": 0.43187533951527735,
    "FBP": 0.06019389866167178,
    "F6P": 2.6879366796632724,
    "G6P": 6.1822543628081625,
    "G1P": 0.35857075303234864,
    "SBP": 1.1860893478540198,
    "S7P": 0.044187268369558365,
    "E4P": 0.2112592088267863,
    "X5P": 0.02098062469006194,
    "R5P": 0.035142545427180046,
    "RUBP": 0.37432944394392,
    "RU5P": 0.0140570180127999,
    "MDA": 0.009653405870311431,
    "ASC": 9.959784342928879,
    "H2O2": 0.0008037536118780133,
    "DHA": 0.030562251200941273,
    "GSH": 9.976204054020393,
    "GSSG": 0.011897972989793133,
        "TR_ox": 0.9,
    "E_inactive": 4.7368421052631575,
}

In [5]:
s = Simulator(m)
s.initialise(y0)
#s.initialise(y0_1000)


pfd=1000
#fcbb=4


s.update_parameter("kMehler", 1.)

#s.update_parameter("fCBB", fcbb)
s.update_parameter("pfd", pfd)


t, y = s.simulate(5000)

Steady State X axis analysis

In [ ]:
variation=np.linspace(0.1,10,50)

original_kcat_dhar=142
original_kcat_mdar= 1080000 / (60 * 60)

s.update_parameter('kcat_DHAR', original_kcat_dhar*10)
s.update_parameter('kcatMDAR', original_kcat_mdar*10)

s.initialise(y0)
s.update_parameter('kMehler', 1)
s.update_parameter('pfd', pfd)
#s.update_parameter("fCBB", fcbb)

s.simulate(8000,steps=10000)
y0=dict(s.get_results_df().iloc[-1,:])

res_dict_kcyc={}


for n in list(s.get_fluxes_dict()):
    res_dict_kcyc.update({n:[]})
for mx in list(dict(s.get_results_df().iloc[-1,:])):
    res_dict_kcyc.update({mx:[]})

for i in variation:
    
    #y0=dict(s.get_results_df().iloc[-1,:])
    
    print(i)
    y0=dict(s.get_results_df().iloc[-1,:])


    s.initialise(y0)
    s.update_parameter('pfd', pfd)
    #s.update_parameter("fCBB", fcbb)

    s.update_parameter('kcyc', i)


    s.simulate_to_steady_state()
    fluxes=s.get_fluxes_dict()
    for j in res_dict_kcyc:
        if j in list(fluxes):
            res_dict_kcyc[j].append(fluxes[j][-1])
        else:
            res_dict_kcyc[j].append(s.get_variable(j)[-1])




In [ ]:
import matplotlib

my_list1=['vPS2','vPS1','vCyc']
my_list2=['vMDAreduct','vRuBisCO','v9']
my_list3=['ATP','NADPH','Fd']


LW=2
fig, (ax1, ax2,ax3) = plt.subplots(3)


for j in my_list1:
    if j == 'vPS2':
        ax1.plot(variation,np.array([2])*res_dict_kcyc[j],label='LEF',linewidth=LW)
    elif j == 'vCyc':
        ax1.plot(variation,np.array([2])*res_dict_kcyc[j],label='CEF',linewidth=LW)
    elif j == 'vPS1':
        ax1.plot(variation,res_dict_kcyc[j],label='rate through PS1',linewidth=LW)
ax1.set_ylabel('Flux (e$^-$mmol/mol Chl/s)',fontsize=8)
ax1.legend(loc='best')
ax1.grid()
ax1.set_xscale('log', basex=2)
ax1.set_xticks([0.125, 0.25, 0.5, 1, 2, 4, 8])
ax1.set_xticklabels( ['-3','-2','-1','0','1','2','3'] )




for j in my_list2:
    if j=='v9':
        ax2.plot(variation,res_dict_kcyc[j],label='vSBPase',linewidth=LW)
    else:
        ax2.plot(variation,res_dict_kcyc[j],label=j,linewidth=LW)
ax2.set_ylabel('Flux (mM/s)')
ax2.legend(loc='best')
ax2.grid()
ax2.set_xscale('log', basex=2)
ax2.set_xticks([0.125, 0.25, 0.5, 1, 2, 4, 8])
ax2.set_xticklabels( ['-3','-2','-1','0','1','2','3'] )

for j in my_list3:
    if j=='ATP':
        ax3.plot(variation,np.array(res_dict_kcyc['ATP'])/m.parameters['APtot'],label=j,linewidth=LW)
    if j=='NADPH':
        ax3.plot(variation,np.array(res_dict_kcyc['NADPH'])/m.parameters['NADPtot'],label=j,linewidth=LW)

    if j=='Fd':
        ax3.plot(variation,np.array(res_dict_kcyc['Fd'])/m.parameters['Fdtot'],label='ox. Fd',linewidth=LW)


ax3.set_ylabel('Flux (mM/s)')
ax3.legend(loc='best')
ax3.grid()
ax3.set_xscale('log', basex=2)
ax3.set_xticks([0.125, 0.25, 0.5, 1, 2, 4, 8])
ax3.set_xticklabels( ['-3','-2','-1','0','1','2','3'] )
ax3.set_xlabel('log$_2$ of kCyc')


In [ ]:
#variation=np.linspace(0.1,70,50)
variation=np.linspace(0.2,4.,50)
y0=dict(s.get_results_df().iloc[-1,:])



#"V9_base": 0.04 * 8
#original_kcat_dhar=142
#original_kcat_mdar= 1080000 / (60 * 60)

#s.update_parameter('kcat_DHAR', original_kcat_dhar*10)
#s.update_parameter('kcatMDAR', original_kcat_mdar*10)

s.initialise(y0)
s.update_parameter('kMehler', 1)
s.update_parameter('pfd', pfd)
#s.update_parameter("fCBB", fcbb)

s.simulate(8000,steps=10000)
y0=dict(s.get_results_df().iloc[-1,:])

res_dict_v9={}


for n in list(s.get_fluxes_dict()):
    res_dict_v9.update({n:[]})
for mx in list(dict(s.get_results_df().iloc[-1,:])):
    res_dict_v9.update({mx:[]})

for i in variation:
    
    #y0=dict(s.get_results_df().iloc[-1,:])
    
    print(i)
    y0=dict(s.get_results_df().iloc[-1,:])


    s.initialise(y0)
    s.update_parameter('pfd', pfd)
    #s.update_parameter("fCBB", fcbb)

    s.update_parameter('V9_base', (0.04 * 8)*i)


    s.simulate_to_steady_state()
    fluxes=s.get_fluxes_dict()
    for j in res_dict_v9:
        if j in list(fluxes):
            res_dict_v9[j].append(fluxes[j][-1])
        else:
            res_dict_v9[j].append(s.get_variable(j)[-1])





In [ ]:
import matplotlib

my_list1=['vPS2','vPS1','vCyc']
my_list2=['vMDAreduct','vRuBisCO','v9']
my_list3=['ATP','NADPH','Fd']


LW=2
fig, (ax1, ax2,ax3) = plt.subplots(3)


for j in my_list1:
    if j == 'vPS2':
        ax1.plot(variation,np.array([2])*res_dict_v9[j],label='LEF',linewidth=LW)
    elif j == 'vCyc':
        ax1.plot(variation,np.array([2])*res_dict_v9[j],label='CEF',linewidth=LW)
    elif j == 'vPS1':
        ax1.plot(variation,res_dict_v9[j],label='rate through PS1',linewidth=LW)
ax1.set_ylabel('Flux (e$^-$mmol/mol Chl/s)',fontsize=8)
ax1.legend(loc='best')
ax1.grid()
ax1.set_xscale('log', basex=2)
ax1.set_xticks([0.25, 0.5, 1, 2, 4])
ax1.set_xticklabels([-2,-1,0,1,2])




for j in my_list2:
    if j=='v9':
        ax2.plot(variation,res_dict_v9[j],label='vSBPase',linewidth=LW)
    else:
        ax2.plot(variation,res_dict_v9[j],label=j,linewidth=LW)
ax2.set_ylabel('Flux (mM/s)')
ax2.legend(loc='best')
ax2.grid()
ax2.set_xscale('log', basex=2)
ax2.set_xticks([0.25, 0.5, 1, 2, 4])
ax2.set_xticklabels([-2,-1,0,1,2])
for j in my_list3:
    if j=='ATP':
        ax3.plot(variation,np.array(res_dict_v9['ATP'])/m.parameters['APtot'],label=j,linewidth=LW)
    if j=='NADPH':
        ax3.plot(variation,np.array(res_dict_v9['NADPH'])/m.parameters['NADPtot'],label=j,linewidth=LW)

    if j=='Fd':
        ax3.plot(variation,np.array(res_dict_v9['Fd'])/m.parameters['Fdtot'],label='ox. Fd',linewidth=LW)


ax3.set_ylabel('Flux (mM/s)')
ax3.legend(loc='best')
ax3.grid()
ax3.set_xscale('log', basex=2)
ax3.set_xticks([0.25, 0.5, 1, 2, 4])
ax3.set_xticklabels([-2,-1,0,1,2])
ax3.set_xlabel('log$_2$ of kSBPase')



Flux Control Coefficients

In [ ]:
def FCC(Flux, Parameter, var=0.01):
    s.simulate_to_steady_state()
    fluxes=s.get_fluxes_dict()
    F=fluxes[Flux][-1]
    
    s.clear_results()
    
    s.update_parameter(Parameter,m.parameters[Parameter]*1.01)
    s.simulate_to_steady_state()
    fluxes=s.get_fluxes_dict()
    F1=fluxes[Flux][-1]
    

    s.clear_results()
    
    s.update_parameter(Parameter,m.parameters[Parameter]*0.99)
    s.simulate_to_steady_state()
    fluxes=s.get_fluxes_dict()
    F99=fluxes[Flux][-1]
    s.update_parameter(Parameter,m.parameters[Parameter])

    return ((F1-F99)/F)/(0.01*2)


def cCC(Var, Parameter, var=0.01):
    s.simulate_to_steady_state()
    
    F=s.get_variable(Var)[-1]
    
    s.clear_results()
    
    s.update_parameter(Parameter,m.parameters[Parameter]*1.01)
    s.simulate_to_steady_state()
    F1=s.get_variable(Var)[-1]
    

    s.clear_results()
    
    s.update_parameter(Parameter,m.parameters[Parameter]*0.99)
    s.simulate_to_steady_state()
    F99=s.get_variable(Var)[-1]
    s.update_parameter(Parameter,m.parameters[Parameter])

    return ((F1-F99)/F)/(0.01*2)


Parameter_of_Interest=['PSIItot','PSItot','kCytb6f','kcyc','kMehler','V1_base','V6_base','V9_base','kcat_DHAR','kcatMDAR']

Reaction_of_Interest=['vPS2','vPS1','vB6f','vMehler','vCyc','vRuBisCO','vFBPase','v9','vMDAreduct','vDHAR']


Parameter_of_Interest_Names=['PSIItot','PSItot','kCytb6f','kcyc','kMehler','kRuBisCO','kFBPase','kSBPase','kDHAR','kMDAR']

Reaction_of_Interest_Names=['vPS2','vPS1','vB6f','vMehler','vCyc','vRuBisCO','vFBPase','vSBPase','vMDAreduct','vDHAR']

In [ ]:
from matplotlib import colors


class MidpointNormalize(colors.Normalize):
	"""
	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

	e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
	"""
	def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
		self.midpoint = midpoint
		colors.Normalize.__init__(self, vmin, vmax, clip)

	def __call__(self, value, clip=None):
		# I'm ignoring masked values and all kinds of edge cases to make a
		# simple example...
		x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
		return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

In [ ]:
y0=dict(s.get_results_df().iloc[-1,:])


s.initialise(y0)
s.update_parameter('kMehler', 1)
#s.update_parameter('fCBB', 4)
s.update_parameter('pfd', 1000)
original_kcat_dhar=142
original_kcat_mdar= 1080000 / (60 * 60)


fcc_result=[]
for i in Reaction_of_Interest:
    print(i)
    res1=[]
    for j in Parameter_of_Interest:
        #print(j)
        cc=FCC(i,j)
        res1.append(cc)
    fcc_result.append(res1)

In [ ]:

elev_min=1
elev_max=-2
mid_val=0


matrix=np.array(fcc_result)
fig, ax = plt.subplots()
im = ax.imshow(matrix,interpolation='nearest', aspect='auto', cmap='RdBu_r',clim=(elev_min, elev_max), norm=MidpointNormalize(midpoint=mid_val,vmin=elev_min, vmax=elev_max))

# We want to show all ticks...
ax.set_xticks(np.arange(len(Parameter_of_Interest_Names)))
ax.set_yticks(np.arange(len(Reaction_of_Interest_Names)))
# ... and label them with the respective list entries
ax.set_xticklabels(Parameter_of_Interest_Names,fontsize=8)
ax.set_yticklabels(Reaction_of_Interest_Names,fontsize=10)
fig.colorbar(im,ax=ax,label='Flux Control Coefficient')
fig.figsize=(80,80)
plt.savefig("Users/nima/FCC.svg")



plt.show()

Surface plots

In [6]:
s.initialise(y0)
s.update_parameter('kMehler', 1)
#s.update_parameter('fCBB', fcbb)
s.update_parameter('pfd', 100)
#original_kcat_dhar=142
#original_kcat_mdar= 1080000 / (60 * 60)

#s.update_parameter('kcat_DHAR', original_kcat_dhar*10)
#s.update_parameter('kcatMDAR', original_kcat_mdar*10)
s.simulate(2000)
y0=dict(s.get_results_df().iloc[-1,:])

pfd_room=np.linspace(50.,1000,7)
#cyc1=np.linspace(0.1,1,4)
#cyc2=np.linspace(1,10,8)
#cyc_room=np.linspace(0.1,10,8)
cyc_room=np.geomspace(0.1, 10, 7)

surface_mdar=[]
surface_ps1=[]
surface_ps2=[]
surface_fd=[]
surface_ph=[]





for i in pfd_room:
    print('pfd: ',i)
    res_mdar=[]
    res_ps1=[]
    res_ps2=[]
    res_fd=[]
    res_ph=[]


    for j in cyc_room:
        print('cyc: ',j)
        y0=dict(s.get_results_df().iloc[-1,:])
        s.update_parameter('pfd', i)
        s.update_parameter('kcyc', j)
#        s.simulate_to_steady_state(atol=1e-14,rtol=1e-14)
        s.simulate_to_steady_state()


        loop_res=s.get_full_results_dict()
        fluxes=s.get_fluxes_dict()
        res_mdar.append(fluxes['vMDAreduct'][-1])
        res_ps1.append(fluxes['vPS1'][-1])
        res_ps2.append(fluxes['vPS2'][-1])
        res_fd.append( ((loop_res['Fd'][-1])/s.model.parameters['Fdtot']) )
        res_ph.append(loop_res['pH'][-1])


    surface_mdar.append(res_mdar)
    surface_ps1.append(res_ps1)
    surface_ps2.append(res_ps2)
    surface_fd.append(res_fd)
    surface_ph.append(res_ph)



pfd:  50.0
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1.0
cyc:  2.1544346900318834
cyc:  4.6415888336127775
cyc:  10.0
pfd:  208.33333333333334
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1.0
cyc:  2.1544346900318834
cyc:  4.6415888336127775
cyc:  10.0
pfd:  366.6666666666667
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1.0
cyc:  2.1544346900318834
cyc:  4.6415888336127775
cyc:  10.0
pfd:  525.0
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1.0
cyc:  2.1544346900318834
cyc:  4.6415888336127775
cyc:  10.0
pfd:  683.3333333333334
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1.0
cyc:  2.1544346900318834
cyc:  4.6415888336127775
cyc:  10.0
pfd:  841.6666666666667
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1.0
cyc:  2.1544346900318834
cyc:  4.6415888336127775
cyc:  10.0
pfd:  1000.0
cyc:  0.1
cyc:  0.21544346900318834
cyc:  0.46415888336127786
cyc:  1

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
X,Y = np.meshgrid(cyc_room,pfd_room)
ax.plot_surface(X, Y, np.array(surface_mdar),cmap='RdYlBu_r', edgecolor='none')
#ax.set_title('Cyclic electron flux over Mehler activity and light intensity')
ax.set_xlabel('kCyc')
ax.set_ylabel('PFD')
ax.set_zlabel('vMDAreduct')



import matplotlib.cm as cm
am = cm.ScalarMappable(cmap=cm.RdYlBu_r)
am.set_array(np.array(surface_mdar))
fig.colorbar(am,ax=ax)
plt.show()

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
X,Y = np.meshgrid(cyc_room,pfd_room)
ax.plot_surface(X, Y, np.array(surface_ps1),cmap='RdYlBu_r', edgecolor='none')
#ax.set_title('Cyclic electron flux over Mehler activity and light intensity')
ax.set_xlabel('kCyc')
ax.set_ylabel('PFD')
ax.set_zlabel('vPS1')



import matplotlib.cm as cm
am = cm.ScalarMappable(cmap=cm.RdYlBu_r)
am.set_array(np.array(surface_ps1))
fig.colorbar(am,ax=ax)
plt.show()

In [9]:
LEF_array=np.array([2])*np.array(surface_ps2)






from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
X,Y = np.meshgrid(cyc_room,pfd_room)
ax.plot_surface(np.log2(X), Y, LEF_array,cmap='RdYlBu_r', edgecolor='none')
#ax.set_title('Cyclic electron flux over Mehler activity and light intensity')
ax.set_xlabel('log$_2$ of kCyc')
ax.set_ylabel('PFD')
ax.set_zlabel('LEF Flux (e$^-$mmol/mol Chl/s)')



import matplotlib.cm as cm
am = cm.ScalarMappable(cmap=cm.RdYlBu_r)
am.set_array(LEF_array)
fig.colorbar(am,ax=ax)
plt.show()

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
X,Y = np.meshgrid(cyc_room,pfd_room)
ax.plot_surface(X, Y, np.array(surface_fd),cmap='RdYlBu_r', edgecolor='none')
#ax.set_title('Cyclic electron flux over Mehler activity and light intensity')
ax.set_xlabel('kCyc')
ax.set_ylabel('PFD')
ax.set_zlabel('Fd')



import matplotlib.cm as cm
am = cm.ScalarMappable(cmap=cm.RdYlBu_r)
am.set_array(np.array(surface_fd))
fig.colorbar(am,ax=ax)
plt.show()

In [ ]:
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
X,Y = np.meshgrid(cyc_room,pfd_room)
ax.plot_surface(X, Y, np.array(surface_ph),cmap='RdYlBu_r', edgecolor='none')
#ax.set_title('Cyclic electron flux over Mehler activity and light intensity')
ax.set_xlabel('kCyc')
ax.set_ylabel('PFD')
ax.set_zlabel('pH')



import matplotlib.cm as cm
am = cm.ScalarMappable(cmap=cm.RdYlBu_r)
am.set_array(np.array(surface_ph))
fig.colorbar(am,ax=ax)
plt.show()

In [ ]:
surface_ph

Steady state analysis, look at redox carriers

surface plot with ps1 ps2

steady state analysis with oxygen concentration


In [ ]:
s.model.parameters['Fdtot']

Here, we simulate the dependency of the system on PFD and oxygen concentration

-Representative Variables statt PGA Fd rein tun, und poolkonzentrationen 0-1 schicken
-log2 x achse versuchen für parameters
-FCC weiß und rot blau andersrum
-Surface plot mit pH auslesen

In [ ]:
np.log2(10)